# 1. Introduction


Accelerometers have recently become one of the most ubiquitous electromechanical sensors due to their presence in virtually all modern smartphones. The most basic application therein is to orient the display based on rotation of the handset, but more sophisticated devices are able to track user movement in three axes. Device applications such as step counters may seek to classify this information to the user's benefit.

The data examined here is raw triaxial accelerometer output sampled at 10Hz. CSV formatted lines contain approximately 6 minutes of measured activity preclassified into one of four activity types: 1) standing, 2) walking, 3) going downstairs, and 4) going upstairs. An activity classification (1,2,3 or 4) is provided once every 10 samples (1 second).

A further 2 minutes of activity remains unclassified. The goal of this work is to train a classification model using the preclassified data and use it to predict the activity type represented by the measurements. To this end, the training dataset is first translated into a pandas dataframe and then explored in both 3D space and in the time domain. After trimming and de-trending the data, statistical features are taken from windowed subsets of the time series of each axis and appended to the dataframe. A random forest classifier is trained on this set of features and used to predict an activity type for each sample in the unclassified test data. Finally, a label is assigned according to the most prevalent prediction in each group of 10 samples.


# 2. Methods

## 2.1. Initial data import and cleaning

It makes sense to use a pandas dataframe to have a look at the training dataset and then work with it in python.

In [1]:
# Import some useful modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# With the CSV data in the working directory, read it into a dataframe and display a few rows
train_data = pd.read_csv('train_time_series.csv')
train_data.head()

,Unnamed: 0,timestamp,UTC time,accuracy,x,y,z
0,20586,1565109930787,2019-08-06T16:45:30.787,unknown,-0.006485,-0.934860,-0.069046
1,20587,1565109930887,2019-08-06T16:45:30.887,unknown,-0.066467,-1.015442,0.089554
2,20588,1565109930987,2019-08-06T16:45:30.987,unknown,-0.043488,-1.021255,0.178467
3,20589,1565109931087,2019-08-06T16:45:31.087,unknown,-0.053802,-0.987701,0.068985
4,20590,1565109931188,2019-08-06T16:45:31.188,unknown,-0.054031,-1.003616,0.126450


In [3]:
# Pull the classification labels into another dataframe
train_labels = pd.read_csv('train_labels.csv')
train_labels.head()

,Unnamed: 0,timestamp,UTC time,label
0,20589,1565109931087,2019-08-06T16:45:31.087,1
1,20599,1565109932090,2019-08-06T16:45:32.090,1
2,20609,1565109933092,2019-08-06T16:45:33.092,1
3,20619,1565109934094,2019-08-06T16:45:34.094,1
4,20629,1565109935097,2019-08-06T16:45:35.097,1


In [4]:
# Check to see what's in the accuracy column of the training set
train_data.accuracy.unique()

array(['unknown'], dtype=object)

Column 'Unnamed: 0' may contain sample numbers but is not useful here and can be removed. A quick check shows there is no useful information in the 'accuracy' column of the test data either. The UTC time column reveals that the data is sampled at 10Hz, but this is also evident from the timestamp column, which is easier to work with. 

Putting the data import and trimming into a function means it can be used for the test data later. It may be useful to merge the training label to the training dataset at each corresponding timestamp, resulting in a combined dataset. The timestamp column will serve as the new index.

In [5]:
def trim(label_path, data_path):
    """
    A function which takes the relative paths of a CSV dataset and corresponding label set and returns a combined
    pandas dataframe indexed by sample timestamp. Unused columns are removed leaving x,y,z coordinates and label.
    """
    # Read the files into dataframes
    data = pd.read_csv(data_path)
    labels = pd.read_csv(label_path)
    # Drop unused columns
    data.drop(['Unnamed: 0', 'UTC time', 'accuracy'], 1, inplace=True)
    labels.drop(['Unnamed: 0', 'UTC time'], 1, inplace=True)
    # Join rows at the corresponding timestamp
    combined = data.set_index('timestamp').join(labels.set_index('timestamp'))
    return combined

In [6]:
# Call the above function to get a combined dataframe
combined_train = trim('train_labels.csv', 'train_time_series.csv')
combined_train.tail()

,x,y,z,label
timestamp,,,,
1565110305638,0.024384,-0.710709,0.030304,NaN
1565110305738,0.487228,-1.099136,-0.015213,NaN
1565110305838,0.369446,-0.968506,0.036713,NaN
1565110305939,0.167877,-0.802826,0.049805,NaN
1565110306039,0.689346,-0.991043,0.034973,4.0


Next the NaNs are replaced with appropriate labels so all data can be used in training. It's reasonable to assume the labels are assigned according to the preceding data entries, and since the last timestamp has a label, it makes sense to backfill the NaNs. This method means it shouldn't matter that the first label doesn't have exactly 10 data points behind it.

In [7]:
# Copy labels onto preceding rows
combined_train.label.fillna(method='bfill', inplace=True)

## 2.2. Data visualisation 

In [8]:
# Plot all data points in 3D space to see if they can be separated there
%matplotlib notebook
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
stand = combined_train[combined_train.label==1]
walk = combined_train[combined_train.label==2]
down = combined_train[combined_train.label==3]
up = combined_train[combined_train.label==4]
ax.scatter(stand.x, stand.y, stand.z, color='blue', label='standing')
ax.scatter(walk.x, walk.y, walk.z, color='green', label='walking')
ax.scatter(down.x, down.y, down.z, color='magenta', label='stairs down')
ax.scatter(up.x, up.y, up.z, color='orange', label='stairs up')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.legend()

<IPython.core.display.Javascript object>

The only group that can be easily distinguished in the raw data is the standing (label = 1) points. They are clustered near the origin, as expected. However, it will be difficult to classify the active behaviour types from this view of the data. Better features for modelling can probably be taken from the time series data.

In [9]:
def plot_timeseries(dataframe):
    """
    A function to plot x,y,z data and label against time. Accepts combined dataframe as returned by trim() function.
    NaNs must be filled to view label plot.
    """
    # Add a column with seconds elapsed from the first sample
    t_0 = dataframe.index[0]
    dataframe['t (s)'] = (dataframe.index - t_0)/1000
    # Create figure and subplot objects
    fig = plt.figure()
    x_ax = fig.add_subplot(411)
    y_ax = fig.add_subplot(412)
    z_ax = fig.add_subplot(413)
    lab = fig.add_subplot(414)
    # Plot the time signals
    x_ax.plot(dataframe['t (s)'], dataframe.x)
    y_ax.plot(dataframe['t (s)'], dataframe.y)
    z_ax.plot(dataframe['t (s)'], dataframe.z)
    lab.plot(dataframe['t (s)'], dataframe.label)
    # Configure plot labels
    x_ax.set_title('X', fontsize=8)
    y_ax.set_title('Y', fontsize=8)
    z_ax.set_title('Z', fontsize=8)
    lab.set_title('Activity Label', fontsize=8)
    lab.set_xlabel('Time (seconds)', fontsize=8)
    x_ax.xaxis.set_visible(False)
    y_ax.xaxis.set_visible(False)
    z_ax.xaxis.set_visible(False)

In [10]:
# Use the above function to plot the test data in time
%matplotlib notebook
plot_timeseries(combined_train)

<IPython.core.display.Javascript object>

## 2.3. Processing and modelling 

It's apparent from the figure above that the Y axis acceleration has a significant negative DC offset. Detrending the data before extracting features should improve model accuracy.

In [11]:
from scipy import signal
combined_train.x = signal.detrend(combined_train.x, type='constant')
combined_train.y = signal.detrend(combined_train.y, type='constant')
combined_train.z = signal.detrend(combined_train.z, type='constant')

In [12]:
plot_timeseries(combined_train)

<IPython.core.display.Javascript object>

The acceleration data is now detrended and the 'standing' activity sees all axes zeroed, as expected.

The next step is to try and pull features from the data which can be used to train a classifier. Also, these features must be calculated using an appropriate method. Some other studies have approached similar problems by examining the data in a sequence of subsets or windows, and then calculating some statistical properties within each window [1-4]. 

Yang, Wang, and Chen [4] employ 8 statistical features for each axis in their model. With fewer behaviours to classify, their approach is mimicked here with 6 of these features: mean value, correlation between axes, variance, signal energy, root-mean-square value, and standard deviation. In contrast to cited works, calculations for this model are made for a window centred on each sample rather than spaced windows with a certain percentage of overlap. This is possible due to the relatively small size of the training dataset. 

For window mean, variance, correlation and standard deviation, the built-in pandas.DataFrame.rolling methods are used. Signal energy is taken as the sum of squares of each sample in the window, and root-mean-square is the square root of the sum of squares divided by the window size. 

In [13]:
def get_features(dataframe, window_size):
    """
    Accepts a dataframe formatted as output from trim() with NaNs filled, and window_size of an integer number of
    samples. Centred rolling window calculations appended to dataframe consist of: mean, correlation, variance,
    energy, root-mean-square, and standard deviation in all 3 accelerometer axes.
    window_size/2 samples are padded with the nearest value at the head and tail of the dataframe to compensate for
    the centred window.
    """
    # Calculate rolling mean, correlation between axes, and variance
    dataframe['meanX'] = dataframe.x.rolling(window_size, center=True).mean()
    dataframe['meanY'] = dataframe.y.rolling(window_size, center=True).mean()
    dataframe['meanZ'] = dataframe.z.rolling(window_size, center=True).mean()
    dataframe['corrXY'] = dataframe.x.rolling(window_size, center=True).corr(dataframe.y)
    dataframe['corrXZ'] = dataframe.x.rolling(window_size, center=True).corr(dataframe.z)
    dataframe['corrYZ'] = dataframe.y.rolling(window_size, center=True).corr(dataframe.z)
    dataframe['varX'] = dataframe.x.rolling(window_size, center=True).var()
    dataframe['varY'] = dataframe.y.rolling(window_size, center=True).var()
    dataframe['varZ'] = dataframe.z.rolling(window_size, center=True).var()
    # Magnitude-based calculations of signal energy and root-mean-square value in the window
    magX = np.square(dataframe.x.abs())
    magY = np.square(dataframe.y.abs())
    magZ = np.square(dataframe.z.abs())
    dataframe['eX'] = magX.rolling(window_size, center=True).sum()
    dataframe['eY'] = magY.rolling(window_size, center=True).sum()
    dataframe['eZ'] = magZ.rolling(window_size, center=True).sum()
    dataframe['rmsX'] = magX.rolling(window_size, center=True).apply(lambda x: np.sqrt(np.sum(x)/window_size))
    dataframe['rmsY'] = magY.rolling(window_size, center=True).apply(lambda x: np.sqrt(np.sum(x)/window_size))
    dataframe['rmsZ'] = magZ.rolling(window_size, center=True).apply(lambda x: np.sqrt(np.sum(x)/window_size))
    # Calculate standard deviation of samples in each window
    dataframe['stdX'] = dataframe.x.rolling(window_size, center=True).std()
    dataframe['stdY'] = dataframe.y.rolling(window_size, center=True).std()
    dataframe['stdZ'] = dataframe.z.rolling(window_size, center=True).std()
    # Pad (window_size/2) NaNs at the head and tail of the dataframe with the first/last valid value respectively
    dataframe.fillna(method='bfill', inplace=True)
    dataframe.fillna(method='ffill', inplace=True)
    
    return dataframe

For this type of data classification, window sizes can range from 2.5s to >16s [5,1]. Ideal window size is tricky and depends on a number of things, including sample rate and duration of activity types. Here, although the sample rate is relatively low (10Hz), the window can't be too large because some activity in the training set only lasts for around 8 seconds. If it is too short, however, the behaviours may not be distinguished. 

A window of 40 samples (4s) of data is chosen to train the model. This is around half of the shortest activity duration.

In [14]:
# Calculate features on the training set with a 40-sample rolling window.
complete_train = get_features(combined_train, 40)
complete_train.tail()

,x,y,z,label,t (s),meanX,meanY,meanZ,corrXY,corrXZ,...,varZ,eX,eY,eZ,rmsX,rmsY,rmsZ,stdX,stdY,stdZ
timestamp,,,,,,,,,,,,,,,,,,,,,
1565110305638,-0.171438,0.294547,-0.054131,4.0,374.851,0.070829,-0.007382,-0.044375,-0.303956,0.418002,...,0.076029,10.259106,9.329485,3.043898,0.506436,0.482946,0.275858,0.507847,0.489042,0.275734
1565110305738,0.291407,-0.093881,-0.099648,4.0,374.951,0.070829,-0.007382,-0.044375,-0.303956,0.418002,...,0.076029,10.259106,9.329485,3.043898,0.506436,0.482946,0.275858,0.507847,0.489042,0.275734
1565110305838,0.173624,0.036750,-0.047722,4.0,375.051,0.070829,-0.007382,-0.044375,-0.303956,0.418002,...,0.076029,10.259106,9.329485,3.043898,0.506436,0.482946,0.275858,0.507847,0.489042,0.275734
1565110305939,-0.027944,0.202430,-0.034630,4.0,375.152,0.070829,-0.007382,-0.044375,-0.303956,0.418002,...,0.076029,10.259106,9.329485,3.043898,0.506436,0.482946,0.275858,0.507847,0.489042,0.275734
1565110306039,0.493525,0.014212,-0.049462,4.0,375.252,0.070829,-0.007382,-0.044375,-0.303956,0.418002,...,0.076029,10.259106,9.329485,3.043898,0.506436,0.482946,0.275858,0.507847,0.489042,0.275734


In [15]:
# Gather the feature column names into a list
covariate_names = (['meanX', 'meanY', 'meanZ', 'corrXY', 'corrXZ', 'corrYZ', 'varX', 'varY', 'varZ', 'eX', 'eY', 'eZ',
                   'rmsX', 'rmsY', 'rmsZ', 'stdX', 'stdY', 'stdZ'])

Recent research suggests that random forest classifiers generally outperform other types for this sort of problem [1-3]. It's therefore decided to import a random forest classifier from the scikit.learn module to use in the model. The classifier is trained with a forest of 200 trees but no limit on branches. The model is also used to determine feature importances and list them from least to greatest.

In [16]:
# Import module and initiate classifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200)

# Fit the model using the feautre columns and their corresponding activity label
clf.fit(complete_train[covariate_names], complete_train['label'])

# Determine the relative importances of features
sorted(list(zip(covariate_names, clf.feature_importances_)), key=lambda tup: tup[1])

[('meanZ', 0.04013603352667989),
 ('eX', 0.04186548207921328),
 ('varY', 0.04290931294167135),
 ('stdY', 0.04388610686544499),
 ('meanY', 0.04438408213156071),
 ('rmsX', 0.04456226328816574),
 ('eY', 0.044846495647011944),
 ('rmsY', 0.045398060582827304),
 ('corrYZ', 0.04580069880250683),
 ('corrXY', 0.04808587320218043),
 ('rmsZ', 0.059124028547121725),
 ('eZ', 0.05973363352436852),
 ('corrXZ', 0.06534779581857961),
 ('varZ', 0.06583376439847295),
 ('stdZ', 0.06814571575737852),
 ('stdX', 0.07491978888862168),
 ('meanX', 0.07899599984775214),
 ('varX', 0.0860248641504424)]

## 2.4. Testing 

To test the model, the testing dataset is first imported and pre-processed in the same way as the training set. The sampled data and labels are joined using the trim() function. The combined set is then detrended and an elapsed time column added. Features can then be taken from the time series using get_features() as above.

Once the feature columns are calculated, the predict method of the trained classifier is called. The model is designed to make a prediction for each data point in the time series. However, a label is only required for every 10 samples. Therefore, the final prediction label is taken by 'popular vote' of predictions within each 10-sample block.

In [17]:
# Import and combine the testing datasets
combined_test = trim('test_labels.csv', 'test_time_series.csv')

# Build an elapsed time column, used for plotting
t_0 = combined_test.index[0]
combined_test['t (s)'] = (combined_test.index - t_0)/1000

# Remove any constant trends from the data
combined_test.x = signal.detrend(combined_test.x, type='constant')
combined_test.y = signal.detrend(combined_test.y, type='constant')
combined_test.z = signal.detrend(combined_test.z, type='constant')

# Extract features in a 4 second window, as with the training set
complete_test = get_features(combined_test, 40)

# Use the classifier to make predictions about the test data
predictions = clf.predict(complete_test[covariate_names])

# Make a list of the mode of predictions for every 10 samples
from collections import Counter
pred_labels = []
for i in range(int(len(predictions)/10)):
    c = Counter(predictions[i*10:i*10+10])
    pred_labels.append(c.most_common(1)[0][0])

# Convert the labels from float to integer
pred_labels = [int(i) for i in pred_labels]

In [18]:
# Append the required prediction labels and save as .csv for report submission
test_labels = pd.read_csv('test_labels.csv')
test_labels.label = pred_labels
test_labels.to_csv('test_labels_complete.csv')

# 3. Results 

## 3.1. Model performance 

The approach detailed above results in activities accurately predicted 60.8% (76/125 seconds) of the time. Although not entirely useless, the model performance is underwhelming. Plotting the time series and prediction labels may indicate what could have been improved.

In [19]:
# Fill the combined test label column with all predictions
combined_test['label'] = predictions

# Plot the accelerometer test data and predicted labels
plot_timeseries(combined_test)

<IPython.core.display.Javascript object>

From the time series plot above, it seems the classifier still struggles to separate walking (2) and stairs down/up (3,4) activities. Judging by the consistent run of stairs up (4) and standing (1) classifications from 7-26s and 34-54s respectively, it's possible that these activities have better recognition.

In [20]:
# Plot the model's final predictions in time
fig = plt.figure()
ax = fig.add_subplot()
ax.plot(np.arange(125), pred_labels)
ax.set_xlabel('time (s)')
ax.set_ylabel('activity label')
ax.set_yticklabels('[1 2 3 4]')
ax.set_title('Final predicted labels')

<IPython.core.display.Javascript object>

<ipython-input-20-a8e42d96c08e>:7: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels('[1 2 3 4]')


Text(0.5, 1.0, 'Final predicted labels')

Looking at the final per-second predictions above, it's evident the model experiences difficulties in the second minute of the test set. Even without knowing the actual test labels, many predictions are clearly not indicative of a natural activity progression and must be in error. The jumps from walking (2) to stairs down/up (3,4) suggest that more or better features are needed to distinguish these activity types. 

## 3.2. Possible improvements 

##### 1) Using the training set to experiement with model parameters 

Taking a subset of the training data with known outcome would have been the best testing method, instead of checking the unknown test data with submission attempts. Unfortunately realised too late.

##### 2) More pre-processing

The acceleration data was detrended before use, but other approaches incorporated filtering of the raw data to remove high frequency transients. Further to that, the standing (1) activity might have been pre-classified from the more dynamic activities through mean values alone.

##### 3) Improved features

Revisiting the feature importances at the end of section 2.3, the most important feature (meanX) has a relative importance of only 0.0883. More rigourous feature engineering might have improved accuracy, such as incorporating jerk (derivative of acceleration) and statistical dispersion, or more frequency-based features such as discrete fast Fourier transform.

# 4. Conclusion 

The aim of this work has been to use python to train and test a classification model for four types of human behaviour based on smartphone accelerometer data.
To do this, the data was converted to pandas DataFrame for easy manipulation. After combining the raw data and known classification labels, the data was explored in 3D space and the time domain to determine how to approach to the problem.

Based on this and some research, it was decided to calculate statistical features from rolling four-second windows on each accelerometer axis. A function was written to accomplish this. Six features per axis were calculated and appended to the dataframe: mean value, correlation between axes, variance, signal energy, root-mean-square value, and standard deviation. This resulted in 18 columns of data to be used as covariates to train a random forest classifier.

After importing and processing the test dataset in the same way as the training set, the trained classifier was used to predict activity labels for each test sample. The most common prediction in each set of 10 samples was then taken as the final prediction. 

With a prediction accuracy of just 60%, the model has room for improvement. The predictions were plotted against time and seemed to reveal difficulties in distinguishing walking flat from walking downstairs, especially in the second half of the test data.

Finally, mistakes were reflected upon and some suggestions made for improvement, such as data filtering and a more robust and refined feature set.

# References 

[1] J. Tatler, P. Cassey, and T.A.A. Prowse, 2018. High accuracy at low frequency: detailed behavioural classification from accelerometer data. Journal of Experimental Biology 2018 221: jeb184085, doi: 10.1242, 29 November 2018. Available at: https://jeb.biologists.org/content/221/23/jeb184085

[2] P.Casale, O. Pujol, and P. Radeva. Physical Activity Recognition fromAccelerometer Data using a Wearable Device. University of Barcelona. Available at: https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.299.8313&rep=rep1&type=pdf

[3] S. Mehrang, J. Pietilä, and I. Korhonen, 2018. An Activity Recognition Framework Deploying the Random Forest Classifier and A Single Optical Heart Rate Monitoring and Triaxial Accelerometer Wrist-Band. Sensors (Basel) 2018 Feb; 18(2): 613,  doi: 10.3390/s18020613, 22 February 2018. Available at: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5856093/

[4] J. Yang, J. Wang, Y. Chen, 2008. Using acceleration measurements for activity recognition: An effectivelearning algorithm for constructing neural classifiers. Pattern Recognition Letters 29 (2008) 2213–2220, doi:10.1016/j.patrec.2008.08.002, 14 August 2008. Available at: https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.617.5920&rep=rep1&type=pdf

[5] D. Anguita1, A. Ghio1, L. Oneto, X. Parra, and J.L. Reyes-Ortiz, 2013.A Public Domain Dataset for Human ActivityRecognition Using Smartphones. ESANN 2013 proceedings, European Symposium on Artificial Neural Networks, Computational  Intelligence and Machine Learning.  Bruges (Belgium), 24-26 April 2013, i6doc.com publ., ISBN 978-2-87419-081-0. Available at: http://www.i6doc.com/en/livre/?GCOI=28001100131010